# Density Matrix Embedding Theory (DMET)

## Table of contents:
* [1. Introduction](#1)
* [2. Theory of DMET](#2)
* [3. First example: DMET-CCSD on Butane](#3)
* [4. Second example: DMET-VQE on an hydrogen ring](#4)
* [5. DMET features](#5)
* [6. Closing words](#6)

Before the introduction, let's do all imports needed in this notebook.

In [1]:
# Import for a pretty jupyter notebook...
import json

# Molecule definition.
from pyscf import gto

# The minimal import for DMET.
from qsdk.problem_decomposition.dmet.dmet_problem_decomposition import DMETProblemDecomposition
# Ability to change localization method.
from qsdk.problem_decomposition.dmet.dmet_problem_decomposition import Localization
# Use for VQE ressources estimation vs DMET.
from qsdk.electronic_structure_solvers.vqe_solver import VQESolver
# Use for comparaison.
from qsdk.electronic_structure_solvers import FCISolver

/home/alex/VirtEnvs/qsdk/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


## 1. Introduction <a class="anchor" id="1"></a>
One of the main objectives of quantum chemistry calculations in the area of materials science is to solve the electronic structure problem, $H\Psi=E\Psi$, as accurately as possible, in order to accelerate the materials design process. In the first example, the butane molecule is shown as an example. 

<img src="img/exact.png" alt="exact" width="200" />

The computational cost for performing accurate calculations of the electronic structure of molecules, however, is usually very expensive. For example, the cost of performing the full CI calculation scales exponentially on a classical computer as the size of the system increases. Therefore, when we target large-sized molecules, those relevant for industry problems, it becomes essential to employ an appropriate strategy for reducing the computational cost. The difficulty is to employ a strategy that consolidate accuracy while reducing computational costs when performing electronic structure calculations. Next, we will developp one of such strategy called Density matrix embedding theory (DMET).

## 2. Theory of DMET <a class="anchor" id="2"></a>

The idea is to decompose a molecular system into its constituent fragments and its environment. Each fragment are treated independently and recombined at the end to recover the full molecular energy. This has the advantages of cutting down the maximal computational complexity depending on the biggest fragment and being naturally implemented on parrallel computer. On the other side, the fragmenting comes at the cost of reducing accuraccy of the computation.

First, the environment is calculated using a less-accurate method than will be used to calculate the electronic structure of a fragment. Then, the electronic structure problem for a given fragment is solved to a high degree of accuracy, which includes the quantum mechanical effects of the environment. The quantum mechanical description is updated (i.e., solved iteratively as shown below) by incorporating the just-performed highly accurate calculation. In the following schematic illustration, the molecule shown above is decomposed into fragments. Each molecular fragment CH$_\mathrm{3}$ and CH$_\text{2}$ are the fragments chosen for the electronic structure calculation, with the rest of the molecular system being the surrounding environment.

<img src="img/iterations.png" alt="iterations" width="600" />

One successful decomposition approach is the DMET method. The DMET method decomposes a molecule into fragments, and each fragment is treated as an open quantum system that is entangled with each of the other fragments, all taken together to be that fragment's surrounding environment (or "bath"). In this framework, the electronic structure of a given fragment is obtained by solving the following Hamiltonian, by using a highly accurate quantum chemistry method, such as the full CI method or a coupled-cluster method.

$$ H_{I}=\sum^{\text{frag}+\text{bath}}_{rs}  \left[ h_{rs} + \sum_{mn} \left[ (rs|mn) - (rn|ms) \right] D^{\text{(mf)env}}_{mn} \right] a_{r}^{\dagger}a_{s} + \sum_{pqrs}^{\text{frag}+\text{bath}} (pq|rs) a_{p}^{\dagger}a_{r}^{\dagger}a_{s}a_{q} - \mu\sum_{r}^{\text{frag}} a_{r}^{\dagger}a_{r} $$

The expression $\sum_{mn} \left[ (rs|mn) - (rn|ms) \right] D^{\text{(mf)env}}_{mn}$ describes the quantum mechanical effects of the environment on the fragment, where $D^{\text{(mf)env}}_{mn}$ is the mean-field electronic density obtained by solving the Hartree&ndash;Fock equation. The quantum mechanical effects from the environment are incorporated through the one-particle term of the Hamiltonian. The extra term $\mu\sum_{r}^{\text{frag}} a_{r}^{\dagger}a_{r}$ ensures, through the adjustment of $\mu$, that the number of electrons in all of the fragments, taken together, becomes equal to the total number of electrons in the entire system.

## 3. First example: DMET-CCSD on Butane <a class="anchor" id="3"></a>

The first example will show how to partition the system into fragments. A different method for solving electronic structures can be chosen for each fragment, but here we will stick with CCSD. The first thing to do is building the butane PySCF molecule object.

In [2]:
butane = """
C   2.142   1.395  -8.932
H   1.604   0.760  -8.260              
H   1.745   2.388  -8.880
H   2.043   1.024  -9.930
C   3.631   1.416  -8.537
H   4.169   2.051  -9.210
H   3.731   1.788  -7.539             
C   4.203  -0.012  -8.612
H   3.665  -0.647  -7.940
H   4.104  -0.384  -9.610
C   5.691   0.009  -8.218  
H   6.088  -0.983  -8.270
H   5.791   0.381  -7.220
H   6.230   0.644  -8.890
"""

# Building the PySCF object.
mol_butane = gto.Mole()
mol_butane.atom = butane
mol_butane.basis = "minao"
mol_butane.charge = 0
mol_butane.spin = 0
mol_butane.build()

The options for the DMET decomposition method are stored in a python dictionary.
* **molecule**: The PySCF molecule object.
* **fragment_atoms**: List for the number of atoms in each fragment. Each atoms are chosen with their order in the coordinates definition (variable `butane`). Also, the sum of all number must be equal to total number of atom in the molecule.
* **fragment_solvers**: A string or a list of string representing the solver for each fragment. The number of items in the list must be equal to the number of fragment. There is one exception: when a single string is defined, the solver is the same for all fragments.
* **verbose**: Activate verbose output.

The next step `dmet.build()` ensures that the fragments and envrionement orbitals are defined according to the electron localization scheme.

In [3]:
options_butane_dmet = {"molecule": mol_butane,
                       # Fragment definition = CH3, CH2, CH2, CH3
                       "fragment_atoms": [4, 3, 3, 4],
                       "fragment_solvers": "ccsd",
                       "verbose": True
                       }

dmet_butane = DMETProblemDecomposition(options_butane_dmet)
dmet_butane.build()

Finally, we can call the `dmet_butane.simulate()` method to do the computation.

In [4]:
energy_butane_dmet = dmet_butane.simulate()
print(f"DMET energy (hartree): \t {energy_butane_dmet}")

 	Iteration =  1
 	----------------
 
		Fragment Number : #  1
		------------------------
		Fragment Energy                 =    -72.0540582760
		Number of Electrons in Fragment =     16.0000000000

		Fragment Number : #  2
		------------------------
		Fragment Energy                 =    -72.4095411539
		Number of Electrons in Fragment =     14.0000000000

		Fragment Number : #  3
		------------------------
		Fragment Energy                 =    -72.4172977137
		Number of Electrons in Fragment =     14.0000000000

		Fragment Number : #  4
		------------------------
		Fragment Energy                 =    -72.0631745662
		Number of Electrons in Fragment =     16.0000000000

 	Iteration =  2
 	----------------
 
		Fragment Number : #  1
		------------------------
		Fragment Energy                 =    -72.0544828916
		Number of Electrons in Fragment =     16.0000000000

		Fragment Number : #  2
		------------------------
		Fragment Energy                 =    -72.4103731263
		Number of E

As seen below, the correlation energy $E_{corr} = E_{DMET}-E_{HF}$ retrieved from this calculation is significant. 

In [5]:
energy_butane_hf = dmet_butane.mean_field.e_tot
energy_corr_butane = abs(energy_butane_dmet - energy_butane_hf)

print(f"Correlation energy (hartree): \t {energy_corr_butane}")
print(f"Correlation energy (kcal/mol): \t {627.5*energy_corr_butane}")

Correlation energy (hartree): 	 0.2645638723222419
Correlation energy (kcal/mol): 	 166.01382988220678


## 4. Second example: DMET-VQE on an hydrogen ring <a class="anchor" id="4"></a>

### 4.1 Why not only using VQE?

We saw in the last section that the computation of DMET fragments is able to get back a significant amount of correlation energy. A valid question that can be asked is: "Why can't we just do the computation on the big molecule?". As stated earlier, DMET breaks the system into its constituents. People who want to do quantum compution on NISQ devices can benefit from this scheme. In fact, performing VQE on an hydrogen ring is beyond the capability of current quantum computer.

We have selected a ring of 10 hydrogen atoms as a simple example of a molecular system. The distance between adjacent hydrogen atoms has been set to 1$~$Å.

In [6]:
H10="""
H          1.6180339887          0.0000000000          0.0000000000
H          1.3090169944          0.9510565163          0.0000000000
H          0.5000000000          1.5388417686          0.0000000000
H         -0.5000000000          1.5388417686          0.0000000000
H         -1.3090169944          0.9510565163          0.0000000000
H         -1.6180339887          0.0000000000          0.0000000000
H         -1.3090169944         -0.9510565163          0.0000000000
H         -0.5000000000         -1.5388417686          0.0000000000
H          0.5000000000         -1.5388417686          0.0000000000
H          1.3090169944         -0.9510565163          0.0000000000
"""

mol_h10 = gto.Mole()
mol_h10.atom = H10
mol_h10.basis = "minao"
mol_h10.charge = 0
mol_h10.spin = 0
mol_h10.build()

We specifiy some standard parameters to show our point. Please note that other encoding could reduce the required resources, but resources would still be too much for current hardware.

In [7]:
options_h10_vqe = {"molecule": mol_h10,
                   "qubit_mapping": "jw",
                   "verbose": False
                   }
vqe_h10 = VQESolver(options_h10_vqe)
vqe_h10.build()

Here are some resources estimation that would be needed for a full VQE calculation.

In [8]:
resources_h10_vqe = vqe_h10.get_resources()
print(json.dumps(resources_h10_vqe, indent=2))

{
  "qubit_hamiltonian_terms": 4435,
  "circuit_width": 20,
  "circuit_gates": 50278,
  "circuit_2qubit_gates": 32704,
  "circuit_var_gates": 1996,
  "vqe_variational_parameters": 350
}


### 4.2 DMET-VQE

Here, we demonstrate how to perform  DMET-VQE calculations using qSDK package. The aim is to obtain improved results (vs HF energy) when compairing to the Full CI method (without using problem decomposition) and also using a quantum algorithm (VQE).

In [19]:
options_h10_dmet = {"molecule": mol_h10,
                    "fragment_atoms": [1]*10,
                    "fragment_solvers": "vqe",
                    "verbose": False
                    }

dmet_h10 = DMETProblemDecomposition(options_h10_dmet)
dmet_h10.build()

The `dmet.build()` method creates fragments (10) from the H10 molecule. When we decompose the ring of atoms into fragments, one of which includes only one hydrogen atom, the DMET method creates a fragment orbital (left: the single orbital distribution is shown in both pink and blue, with the colours depicting the phases) and the bath orbital (right: the single orbital distribution of the remaining nine hydrogen atoms is shown in both pink and blue, with the colours depicting the phases). 

<img src="img/frag_and_bath.png" alt="fragment_and_bath_orbitals" width="450"/>

Ressources estimations is done by calling `dmet_h10.get_resources()`. Here, a list of ten dictionaries is returned and stored in `resources_h10_dmet`. Each dictionary refers to a fragment. As every fragment is the same in this system (a single hydrogen atom), we only print one.

In [20]:
resources_h10_dmet = dmet_h10.get_resources()
print(json.dumps(resources_h10_dmet[0], indent=2))

{
  "qubit_hamiltonian_terms": 27,
  "circuit_width": 4,
  "circuit_gates": 158,
  "circuit_2qubit_gates": 64,
  "circuit_var_gates": 12,
  "vqe_variational_parameters": 2
}


Compared to an all VCQE algorithm, those resources are greatly reduced. Below, `dmet_h10.simulate()` computes the DMET-VQE energy.

In [21]:
dmet_h10.verbose = True
energy_h10_dmet = dmet_h10.simulate()

print(f"DMET energy (hartree): \t {energy_h10_dmet}")

 	Iteration =  1
 	----------------
 
		Fragment Number : #  1
		------------------------


/home/alex/Codes/QEMIST_qSDK/qsdk/toolboxes/ansatz_generator/uccsd.py:61: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if var_params == "ones":
/home/alex/Codes/QEMIST_qSDK/qsdk/toolboxes/ansatz_generator/uccsd.py:63: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif var_params == "random":
/home/alex/Codes/QEMIST_qSDK/qsdk/toolboxes/ansatz_generator/uccsd.py:65: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif var_params == "MP2":


Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.6862371645340293
            Iterations: 5
            Function evaluations: 20
            Gradient evaluations: 5
		Fragment Energy                 =     -1.8000328075
		Number of Electrons in Fragment =      2.0000000000

		Fragment Number : #  2
		------------------------
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.6862371644300924
            Iterations: 5
            Function evaluations: 20
            Gradient evaluations: 5
		Fragment Energy                 =     -1.8001880724
		Number of Electrons in Fragment =      2.0000000000

		Fragment Number : #  3
		------------------------
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.6862371644960765
            Iterations: 5
            Function evaluations: 20
            Gradient evaluations: 5
		Fragment Energy                 =     -1.80003280

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.686032928088384
            Iterations: 5
            Function evaluations: 20
            Gradient evaluations: 5
		Fragment Energy                 =     -1.7998999556
		Number of Electrons in Fragment =      2.0000000000

		Fragment Number : #  4
		------------------------
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.6860329281620883
            Iterations: 5
            Function evaluations: 20
            Gradient evaluations: 5
		Fragment Energy                 =     -1.7997446762
		Number of Electrons in Fragment =      2.0000000000

		Fragment Number : #  5
		------------------------
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.6860329280914481
            Iterations: 5
            Function evaluations: 20
            Gradient evaluations: 5
		Fragment Energy                 =     -1.799899955

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.6861008155867336
            Iterations: 5
            Function evaluations: 20
            Gradient evaluations: 5
		Fragment Energy                 =     -1.7999957345
		Number of Electrons in Fragment =      2.0000000000

		Fragment Number : #  6
		------------------------
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.686100815621602
            Iterations: 5
            Function evaluations: 20
            Gradient evaluations: 5
		Fragment Energy                 =     -1.7999957347
		Number of Electrons in Fragment =      2.0000000000

		Fragment Number : #  7
		------------------------
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.6861008155867416
            Iterations: 5
            Function evaluations: 20
            Gradient evaluations: 5
		Fragment Energy                 =     -1.799995734

A compraison with an FCI calculation is then made.

In [12]:
fci_h10 = FCISolver()
energy_h10_fci = fci_h10.simulate(mol_h10)

print(f"FCI energy (hartree): \t {energy_h10_fci}")

FCI energy (hartree): 	 -5.380926000730881


Lastly, we note that the DMET energy is closer to the FCI than the HF energy.

In [13]:
energy_h10_hf = dmet_h10.mean_field.e_tot
delta_h10_fci_hf = abs(energy_h10_fci - energy_h10_hf)
delta_h10_fci_dmet = abs(energy_h10_fci - energy_h10_dmet)

print(f"Difference FCI vs HF energies (hartree): \t\t {delta_h10_fci_hf}")
print(f"Difference FCI vs DMET-VQE energies (hartree): \t\t {delta_h10_fci_dmet}")
print(f"Difference FCI vs HF energies (kcal/mol): \t\t {627.5*delta_h10_fci_hf}")
print(f"Difference FCI vs DMET-VQE energies (kcal/mol): \t {627.5*delta_h10_fci_dmet}")

Difference FCI vs HF energies (hartree): 		 0.11680533556176087
Difference FCI vs DMET-VQE energies (hartree): 		 0.013393208289180336
Difference FCI vs HF energies (kcal/mol): 		 73.29534806500494
Difference FCI vs DMET-VQE energies (kcal/mol): 	 8.404238201460661


## 5. DMET features<a class="anchor" id="5"></a>

In this section, some DMET features are shown. An four hydrogen atoms system is defined for this purpose.

In [14]:
H4 = """
H 0.7071 0.0 0.0
H 0.0 0.7071 0.0
H -1.0071 0.0 0.0
H 0.0 -1.0071 0.0
"""

mol_h4 = gto.Mole()
mol_h4.atom = H4
mol_h4.basis = "3-21g"
mol_h4.charge = 0
mol_h4.spin = 0
mol_h4.build()

### 5.1 Localization method

Electron localization are used to define the bath and the environment orbitals. There are two options available:
- `Localization.meta_lowdin` (default): Described in Q. Sun et al., JCTC 10, 3784-3790 (2014).
- `Localization.iao`: Described in G. Knizia, JCTC 9, 4834-4843 (2013). This algorithm maps the orbitals to an minao set. So, at least a double zeta basis set must be used with this localization method.

In [15]:
options_h4_dmet = {"molecule": mol_h4,
                   "fragment_atoms": [1]*4,
                   "electron_localization": Localization.iao,
                   "verbose": False
                   }

dmet_h4 = DMETProblemDecomposition(options_h4_dmet)
vars(dmet_h4)

{'molecule': <pyscf.gto.mole.Mole at 0x7f9beeae3820>,
 'mean_field': None,
 'electron_localization': <Localization.iao: 1>,
 'fragment_atoms': [1, 1, 1, 1],
 'fragment_solvers': ['ccsd', 'ccsd', 'ccsd', 'ccsd'],
 'optimizer': <bound method DMETProblemDecomposition._default_optimizer of <qsdk.problem_decomposition.dmet.dmet_problem_decomposition.DMETProblemDecomposition object at 0x7f9beeaf0580>>,
 'initial_chemical_potential': 0.0,
 'solvers_options': [{}, {}, {}, {}],
 'verbose': False,
 'chemical_potential': None,
 'dmet_energy': None,
 'orbitals': None,
 'orb_list': None,
 'orb_list2': None,
 'onerdm_low': None}

### 5.2 Fragment solvers

A list of solvers can be passed to the DMET class. If a single solver is detected, it will be applied to every fragments. Here is an example where the first fragment is solved with VQE, the second one with CCSD, the third one with FCI and the last one with VQE.

In [16]:
options_h4_dmet = {"molecule": mol_h4,
                   "fragment_atoms": [1]*4,
                   "fragment_solvers": ["vqe", "ccsd", "fci", "vqe"],
                   "verbose": False
                   }

dmet_h4 = DMETProblemDecomposition(options_h4_dmet)
vars(dmet_h4)

{'molecule': <pyscf.gto.mole.Mole at 0x7f9beeae3820>,
 'mean_field': None,
 'electron_localization': <Localization.meta_lowdin: 0>,
 'fragment_atoms': [1, 1, 1, 1],
 'fragment_solvers': ['vqe', 'ccsd', 'fci', 'vqe'],
 'optimizer': <bound method DMETProblemDecomposition._default_optimizer of <qsdk.problem_decomposition.dmet.dmet_problem_decomposition.DMETProblemDecomposition object at 0x7f9beeae31c0>>,
 'initial_chemical_potential': 0.0,
 'solvers_options': [{'qubit_mapping': 'jw',
   'initial_var_params': 'ones',
   'verbose': False},
  {},
  {},
  {'qubit_mapping': 'jw', 'initial_var_params': 'ones', 'verbose': False}],
 'verbose': False,
 'chemical_potential': None,
 'dmet_energy': None,
 'orbitals': None,
 'orb_list': None,
 'orb_list2': None,
 'onerdm_low': None}

### 5.3 Initial chemical potential

The DMET optimize a parameter $\mu$, the chemical potential. It ensures that the sum of all fragment electron are consistent with the whole molecule. As it is numerically optimized, the initial value can be important. Here is an example where the `initial_chemical_potential` is set to 0.1.

In [17]:
options_h4_dmet = {"molecule": mol_h4,
                   "fragment_atoms": [1]*4,
                   "initial_chemical_potential" : 0.1,
                   "verbose": False
                   }

dmet_h4 = DMETProblemDecomposition(options_h4_dmet)
vars(dmet_h4)

{'molecule': <pyscf.gto.mole.Mole at 0x7f9beeae3820>,
 'mean_field': None,
 'electron_localization': <Localization.meta_lowdin: 0>,
 'fragment_atoms': [1, 1, 1, 1],
 'fragment_solvers': ['ccsd', 'ccsd', 'ccsd', 'ccsd'],
 'optimizer': <bound method DMETProblemDecomposition._default_optimizer of <qsdk.problem_decomposition.dmet.dmet_problem_decomposition.DMETProblemDecomposition object at 0x7f9beeae3370>>,
 'initial_chemical_potential': 0.1,
 'solvers_options': [{}, {}, {}, {}],
 'verbose': False,
 'chemical_potential': None,
 'dmet_energy': None,
 'orbitals': None,
 'orb_list': None,
 'orb_list2': None,
 'onerdm_low': None}

### 5.3 Solvers options

A list of options can be passed to the solvers. Each element of the list must be consistent with the appropriate value in `fragment_solvers`. If a single dictionary of options is detected, it is applied to all fragment solvers. Here is an example where one want to change the qubit mapping to the Bravyi-Kitaev method when performing VQE.

In [18]:
vqe_options = {"qubit_mapping": "bk"}

options_h4_dmet = {"molecule": mol_h4,
                   "fragment_atoms": [1]*4,
                   "fragment_solvers": "vqe",
                   "solvers_options": vqe_options, 
                   "verbose": False
                   }

dmet_h4 = DMETProblemDecomposition(options_h4_dmet)
vars(dmet_h4)

{'molecule': <pyscf.gto.mole.Mole at 0x7f9beeae3820>,
 'mean_field': None,
 'electron_localization': <Localization.meta_lowdin: 0>,
 'fragment_atoms': [1, 1, 1, 1],
 'fragment_solvers': ['vqe', 'vqe', 'vqe', 'vqe'],
 'optimizer': <bound method DMETProblemDecomposition._default_optimizer of <qsdk.problem_decomposition.dmet.dmet_problem_decomposition.DMETProblemDecomposition object at 0x7f9beeaf0c40>>,
 'initial_chemical_potential': 0.0,
 'solvers_options': [{'qubit_mapping': 'bk'},
  {'qubit_mapping': 'bk'},
  {'qubit_mapping': 'bk'},
  {'qubit_mapping': 'bk'}],
 'verbose': False,
 'chemical_potential': None,
 'dmet_energy': None,
 'orbitals': None,
 'orb_list': None,
 'orb_list2': None,
 'onerdm_low': None}

## 6. Closing words<a class="anchor" id="6"></a>

This concludes our overview of `DMETProblemDecomposition`. There are many flavors of DMET and only one has been discussed here. Here we refer some papers relevant for the reader who wants more details:
- S. Wouters, C.A. Jiménez-Hoyos, Q. Sun, and G.K.L. Chan, J. Chem. Theory Comput. 12, 2706 (2016).
- G. Knizia and G.K.L. Chan, J. Chem. Theory Comput. 9, 1428 (2013).
- G. Knizia and G.K.L. Chan, Phys. Rev. Lett. 109, 186404 (2012).